# Demo Notebook for Sentence Transformer Model Training, Saving and Uploading to OpenSearch
This notebook provides a walkthrough guidance for users use their synthetic queries to fine tune and train a sentence transformer model. In this notebook, you use opensearch_py_ml to accomplish the following:

Step 0: Import packages and set up client

Step 1: Read synthetic queries and train/fine-tune model using a hugging face sentence transformer model

Step 2: (Optional) Save model

Step 3: Upload the model to OpenSearch cluster

## Step 0: Import packages and set up client
Install required packages for opensearch_py_ml.sentence_transformer_model
Install `opensearchpy` and `opensearch-py-ml` through pypi

Please refer https://pytorch.org/ to proper install torch based on your environment setting.  

In [ ]:
# pip install pandas matplotlib numpy torch accelerate sentence_transformers tqdm transformers opensearch-py opensearch-py-ml 

In [2]:
import warnings
warnings.filterwarnings('ignore')
import opensearch_py_ml as oml
from opensearchpy import OpenSearch
from opensearch_py_ml.ml_models import SentenceTransformerModel

In [3]:
# import mlcommon to later upload the model to OpenSearch Cluster
from opensearch_py_ml.ml_commons import MLCommonClient

In [4]:
CLUSTER_URL = 'https://localhost:9200'

In [5]:
def get_os_client(cluster_url = CLUSTER_URL,
                  username='admin',
                  password='admin'):
    '''
    Get OpenSearch client
    :param cluster_url: cluster URL like https://ml-te-netwo-1s12ba42br23v-ff1736fa7db98ff2.elb.us-west-2.amazonaws.com:443
    :return: OpenSearch client
    '''
    client = OpenSearch(
        hosts=[cluster_url],
        http_auth=(username, password),
        verify_certs=False
    )
    return client 

In [6]:
client = get_os_client()

## Step 1: Read synthetic queries and train/fine-tune model using a hugging face sentence transformer model
With a synthetic queries zip file, users can fine tune a sentence transformer model. 

The `SentenceTransformerModel` class will inititate an object for training, exporting and configuring the model. Plese visit the [SentenceTransformerModel](https://opensearch-project.github.io/opensearch-py-ml/reference/api/sentence_transformer.html#opensearch_py_ml.sentence_transformer_model.SentenceTransformerModel) for API Reference . 

The `train` function will import synthestic queries, load sentence transformer example and train the model using a hugging face sentence transformer model. Plese visit the [SentenceTransformerModel.train](https://opensearch-project.github.io/opensearch-py-ml/reference/api/sentence_transformer.html#opensearch_py_ml.sentence_transformer_model.SentenceTransformerModel.train) for API Reference . 

In [7]:
# clean up cache before training to free up spaces
import gc, torch

gc.collect()

torch.cuda.empty_cache()

In [8]:
# initiate SentenceTransformerModel object and run train function
# num_processes should be less than the number of gpus on the users' machine, if None, it will auto apply 
# all the available gpus on the machine 
custom_model = SentenceTransformerModel(folder_path="/Volumes/workplace/upload_content/model_files/", overwrite = True)
training = custom_model.train(read_path = '/Volumes/workplace/upload_content/synthetic_queries.zip',
                        output_model_name = 'test2_model.pt',
                        zip_file_name= 'test2_model.zip',
                        overwrite = False,
                        use_accelerate  = True,  
                        num_machines = 1,
                        num_processes = 1,
                        num_epochs = 1,)

reading synthetic query file: /Volumes/workplace/upload_content/model_files/synthetic_queries/synthetic_queries_batch_3.p

reading synthetic query file: /Volumes/workplace/upload_content/model_files/synthetic_queries/synthetic_queries_batch_0.p

reading synthetic query file: /Volumes/workplace/upload_content/model_files/synthetic_queries/synthetic_queries_batch_1.p

reading synthetic query file: /Volumes/workplace/upload_content/model_files/synthetic_queries/synthetic_queries_batch_2.p

Loading training examples... 



100%|██████████| 243/243 [00:00<00:00, 148986.39it/s]

generated config file: at /Users/dhrubo/.cache/huggingface/accelerate/default_config.yaml

[{'compute_environment': 'LOCAL_MACHINE', 'deepspeed_config': {'gradient_accumulation_steps': 1, 'offload_optimizer_device': 'none', 'offload_param_device': 'none', 'zero3_init_flag': False, 'zero_stage': 2}, 'distributed_type': 'DEEPSPEED', 'downcast_bf16': 'no', 'fsdp_config': {}, 'machine_rank': 0, 'main_process_ip': None, 'main_process_port': None, 'main_training_function': 'main', 'mixed_precision': 'no', 'num_machines': 1, 'num_processes': 1, 'use_cpu': False}]
generating config file for ml common upload: /Users/dhrubo/.cache/huggingface/accelerate/default_config.yaml



Launching training on MPS.
Start training with accelerator...

The number of training epoch are 1

The total number of steps training epoch are 8

Training epoch 0...



100%|██████████| 8/8 [00:16<00:00,  2.06s/it]


Total training time: 17.660495042800903

Preparing model to save...

Model saved to path: /Volumes/workplace/upload_content/model_files/test2_model.pt

model path is:  /Volumes/workplace/upload_content/model_files/test2_model.pt
Zip file name without extension:  test2_model
tokenizer_json_path:  /Volumes/workplace/upload_content/model_files/tokenizer.json
zip file is saved to /Volumes/workplace/upload_content/model_files/test2_model.zip



## Step 2: (Optional) Save model
If following step 1, the model zip will be auto generated, and the print message will indicate the zip file path as shown above. 

But if using other pretrained sentence transformer model from Hugging face, users can use `save_as_pt` function to save a pre-trained sentence transformer model for inferencing or benchmark with other models. 

The `save_as_pt`  function will prepare the model in proper format(Torch Script) along with tokenizers configuration file to upload to OpenSearch. Plese visit the [SentenceTransformerModel.save_as_pt](https://opensearch-project.github.io/opensearch-py-ml/reference/api/sentence_transformer.html#opensearch_py_ml.sentence_transformer_model.SentenceTransformerModel.save_as_pt) for API Reference . 

In [ ]:
# default to download model id, "sentence-transformers/msmarco-distilbert-base-tas-b" from hugging face 
# and output a model in a zip file containing model.pt file and tokenizers.json file. 
pre_trained_model = SentenceTransformerModel(folder_path = '/Volumes/workplace/upload_content/export_huggingface/', overwrite = True)
pre_trained_model.save_as_pt(sentences = ['today is sunny'])

## Step 3: Upload the model to OpenSearch cluster
After generated a model zip file, the users will need to describe model configuration in a ml-commons_model_config.json file. The `make_model_config_json` function in sentencetransformermodel class will parse the config file from hugging-face config.son file. If users would like to use a different config than the pre-trained sentence transformer, `make_model_config_json` function provide arguuments to change the configuration content and generated a ml-commons_model_config.json file. Plese visit the [SentenceTransformerModel.make_model_config_json](https://opensearch-project.github.io/opensearch-py-ml/reference/api/sentence_transformer.html#opensearch_py_ml.sentence_transformer_model.SentenceTransformerModel.make_model_config_json) for API Reference . 

In general, the ml common client supports uploading sentence transformer models. With a zip file contains model in  Torch Script format, and a configuration file for tokenizers in json format, the `upload_model` function connects to opensearch through ml client and upload the model. Plese visit the [MLCommonClient.upload_model](https://opensearch-project.github.io/opensearch-py-ml/reference/api/ml_commons_upload_api.html#opensearch_py_ml.ml_commons_integration.MLCommonClient.upload_model) for API Reference. 

In [8]:
#users will need to prepare a ml-commons_model_config.json file to config the model, including model name ..
#this is a helpful function in py-ml.sentence_transformer_model to generate ml-commons_model_config.json file
custom_model.make_model_config_json()

reading config file from: /Volumes/workplace/upload_content/model_files/config.json
generating ml-commons_model_config.json file...

{'name': 'sentence-transformers/msmarco-distilbert-base-tas-b', 'version': 1, 'model_format': 'TORCH_SCRIPT', 'model_task_type': 'TEXT_EMBEDDING', 'model_config': {'model_type': 'distilbert', 'embedding_dimension': 768, 'framework_type': 'sentence_transformers', 'all_config': '{"_name_or_path": "/Users/dhrubo/.cache/torch/sentence_transformers/sentence-transformers_msmarco-distilbert-base-tas-b/", "activation": "gelu", "architectures": ["DistilBertModel"], "attention_dropout": 0.1, "dim": 768, "dropout": 0.1, "hidden_dim": 3072, "initializer_range": 0.02, "max_position_embeddings": 512, "model_type": "distilbert", "n_heads": 12, "n_layers": 6, "pad_token_id": 0, "qa_dropout": 0.1, "seq_classif_dropout": 0.2, "sinusoidal_pos_embds": false, "tie_weights_": true, "torch_dtype": "float32", "transformers_version": "4.16.2", "vocab_size": 30522}'}}
ml-commons_m

In [9]:
#connect to ml_common client with OpenSearch client
import opensearch_py_ml as oml
from opensearch_py_ml.ml_commons import MLCommonClient
ml_client = MLCommonClient(client)

In [10]:
# upload model to OpenSearch cluster, using model zip file path and ml-commons_model_config.json file generated above

model_path = '/Volumes/workplace/upload_content/model_files/test2_model.zip'
model_config_path = '/Volumes/workplace/upload_content/model_files/ml-commons_model_config.json'
ml_client.upload_model( model_path, model_config_path, isVerbose=True)

Total number of chunks 27
Sha1 value of the model file:  ec883a9887b9a9a468c6af2fb62a9175057f8c0e522a857a0cc4ec1f1b73733e
Model meta data was created successfully. Model Id:  c61eMYUB385XdR-p0PTN
uploading chunk 1 of 27
{'status': 'Uploaded'}
uploading chunk 2 of 27
{'status': 'Uploaded'}
uploading chunk 3 of 27
{'status': 'Uploaded'}
uploading chunk 4 of 27
{'status': 'Uploaded'}
uploading chunk 5 of 27
{'status': 'Uploaded'}
uploading chunk 6 of 27
{'status': 'Uploaded'}
uploading chunk 7 of 27
{'status': 'Uploaded'}
uploading chunk 8 of 27
{'status': 'Uploaded'}
uploading chunk 9 of 27
{'status': 'Uploaded'}
uploading chunk 10 of 27
{'status': 'Uploaded'}
uploading chunk 11 of 27
{'status': 'Uploaded'}
uploading chunk 12 of 27
{'status': 'Uploaded'}
uploading chunk 13 of 27
{'status': 'Uploaded'}
uploading chunk 14 of 27
{'status': 'Uploaded'}
uploading chunk 15 of 27
{'status': 'Uploaded'}
uploading chunk 16 of 27
{'status': 'Uploaded'}
uploading chunk 17 of 27
{'status': 'Uploaded

'c61eMYUB385XdR-p0PTN'

In [11]:
# Now we can load the uploaded model into memory by using the model id.


ml_client.load_model("c61eMYUB385XdR-p0PTN")

{'task_id': 'dK1gMYUB385XdR-pnfQL', 'status': 'CREATED'}